<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

/var/folders/sp/hjjl899x4js87jvp7rfdpj2w0000gn/T/ipykernel_2098/3849064916.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [6]:
#!pip install --upgrade tensorflow
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65954.046875
Loss after epoch 2: 65933.953125
Loss after epoch 3: 65712.0
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.34375
Loss after epoch 6: 64058.96875
Loss after epoch 7: 64755.40625
Loss after epoch 8: 62585.4375
Loss after epoch 9: 60415.1875
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.0
Loss after epoch 12: 57725.9375
Loss after epoch 13: 56492.6875
Loss after epoch 14: 55842.25
Loss after epoch 15: 55862.8125
Loss after epoch 16: 51696.3125
Loss after epoch 17: 49815.375
Loss after epoch 18: 49568.5
Loss after epoch 19: 48960.125


(156986, 287740)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087130427360535),
 ('sleep', 0.8649648427963257),
 ('help', 0.8594317436218262),
 ('try', 0.8456018567085266),
 ('cry', 0.8387173414230347),
 ('little', 0.827370822429657),
 ('not', 0.8181068301200867),
 ('seems', 0.8175182342529297),
 ('twist', 0.8154944777488708),
 ('peace', 0.8122634887695312)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786577999591827),
 ('our', -0.24495965242385864),
 ('come', -0.2613307535648346),
 ('on', -0.26722919940948486),
 ('bang', -0.2732996344566345),
 ('five', -0.2741815447807312),
 ('six', -0.2774087190628052),
 ('baby', -0.27763256430625916),
 ('work', -0.2790363132953644),
 ('four', -0.27926990389823914)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.978668749332428),
 ('three', 0.9761052131652832),
 ('six', 0.9668912887573242),
 ('two', 0.9565725922584534),
 ('seven', 0.9542326927185059),
 ('sixty', 0.9003374576568604),
 ('one', 0.8165037631988525),
 ('us', 0.7762877941131592),
 ('crying', 0.7727354764938354),
 ('strawberry', 0.7672429084777832)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409599900245667),
 ('buy', 0.9399517774581909),
 ('much', 0.8996836543083191),
 ('hide', 0.8465179204940796),
 ('just', 0.8415131568908691)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario: 
# DA ERROR Y ES DE GUSTO PARA PROBAR QUE LA PALABRA NO ESTA
#w2v_model.wv.most_similar(negative=["diedaa"])

In [19]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736882  0.21460854 -0.05407826  0.06707428  0.13059781 -0.2571417
  0.00711527  0.26744702 -0.03100352  0.25997368  0.21179642 -0.0846577
 -0.26272154  0.30837795 -0.07934512 -0.13357104  0.29946792  0.18062183
 -0.1311193  -0.11857399  0.0346803   0.11255896  0.08478769  0.08090526
 -0.17622426  0.03122874 -0.08725718  0.23746894 -0.11507447 -0.29119375
 -0.03650398 -0.33222464  0.23264216  0.10366732 -0.18365778  0.08299293
  0.14315514 -0.1976166  -0.00757935  0.11195532 -0.10420062 -0.13083553
 -0.03264889 -0.04687171  0.11330322  0.10231242 -0.16201775 -0.10347403
  0.03660134 -0.08524159 -0.3487206  -0.02153995  0.25938615  0.16540702
  0.03108545  0.09505955  0.24453107 -0.09859782  0.18298718  0.06454364
  0.01696082 -0.32592124 -0.04773575 -0.05419112 -0.04338786 -0.09137381
  0.00890292  0.11636212 -0.25662866  0.0542677   0.08916717  0.04540667
  0.20835876 -0.19101627  0.32210258  0.19337815  0.09080074 -0.00362744
 -0.18664072 -0.07053233 -0.12137611 -0.03477536 -0.1

In [20]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0000001192092896),
 ('babe', 0.9132980704307556),
 ('someone', 0.8914985656738281),
 ('nothing', 0.8811895847320557),
 ('need', 0.8758766651153564),
 ("didn't", 0.8689678907394409),
 ("there's", 0.8557928800582886),
 ('you', 0.8447728753089905),
 ('feed', 0.8422091603279114),
 ('somebody', 0.8385541439056396)]

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132980704307556),
 ('someone', 0.8914985656738281),
 ('nothing', 0.8811895847320557),
 ('need', 0.8758766055107117),
 ("didn't", 0.8689678907394409),
 ("there's", 0.8557928800582886),
 ('you', 0.8447728753089905),
 ('feed', 0.8422091603279114),
 ('somebody', 0.8385541439056396),
 ('hope', 0.83614581823349)]

### 5 - Visualizar agrupación de vectores

In [22]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [24]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [25]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.